In [142]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
sys.path.append(os.path.join(os.getcwd(), 'domain'))
from tests.domain.primary.domain.create_domain import DomainTables

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float32'
dim=3
mesh_name = 'tetrahedron.msh'

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'domain', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=4, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 3
path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 2
path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 1
path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 0
====> Start <=====
====> Creating Mesh <=====
gmsh:physical => triangle
gmsh:physical => tetra
node_cellid
Execution time: 0.007462 seconds
cell_cellnid
Execution time: 0.015423 seconds
cell_cellfid and boundary_faces
Execution time: 0.028742 seconds
node_bfid
Execution time: 0.029100 seconds
Start creating sub domains
Parse Input 
Start Creating SubDomains 
Make n part
Start dense to csr
Nb_vertices = 6000
size = 22800
End dense to csr
End METIS_PartGraphKway
End function
Create local cells
Create Physical faces
Create local domains struct
C

In [144]:
import importlib

module = importlib.import_module("helpers.TablesTestTetra3D")
importlib.reload(module)
TablesTestTetra3D = getattr(module, "TablesTestTetra3D")

module = importlib.import_module("helpers.TetraChecker3D")
importlib.reload(module)
TetraChecker3D = getattr(module, "TetraChecker3D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TablesTestTetra3D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = TetraChecker3D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_node_info()
# checker.test_face_info()
checker.summary()
# checker.test_halo_info()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 8 [P] 1 [F] Total 9
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/9: Cell Vertices
[P] 2/9: Cell Halo by node
[P] 3/9: Cell Ghostnid *
* [F] 4/9: Cell Haloghostnid and Haloghostcenter * 1 / 6000 688 Failed
	
	Arrays are not almost equal to 4 decimals
	
	(shapes (0, 3), (7, 3) mismatch)
	 x: array([], shape=(0, 3), dtype=float32)
	 y: array([[ 1.25 , -0.125,  7.875],
	       [-0.25 ,  0.625,  7.875],
	       [ 0.5  , -0.125,  8.25 ],...
[P] 5/9: Cell Number of cells
[P] 6/9: Node Cellid
[P] 7/9: Node Loctoglob
[P] 8/9: Node Halonid
[P] 9/9: Node Number of nodes

---------------------------------------------------------
---------------------------------------------------------
===> 8 [P] 1 [F] Total 9


False

In [131]:
p = 0
i = 0
loctoglob = domain_tables.d_cell_loctoglob[p]
halonid = test_tables.l_cell_ghostnid[39]
print(halonid)

[ 28  29  30  32  57  58 294 295 296  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1   9]


In [143]:
p = 0
i = 2
d_cell_ghostnid = domain_tables.d_cell_ghostnid[p]
d_face_ghostcenter = domain_tables.d_face_ghostcenter[p]
c_ghostnid = d_cell_ghostnid[i]
print(c_ghostnid)
c_ghostn_center = d_face_ghostcenter[c_ghostnid[0:c_ghostnid[-1]]]
print(c_ghostn_center)


[  2 441   5  65  73  15  19 444 456   0   0   0   0   0   0   0   0   9]
[[ 0.5        -0.125       9.75       -2.2926512 ]
 [ 1.25       -0.125       9.375      -2.757817  ]
 [-0.25        0.25       10.125      -0.08333334]
 [-0.25        0.75        8.625      -1.5       ]
 [-0.25        0.625       9.375      -0.4166667 ]
 [ 0.25       -0.125      10.875      -2.0714288 ]
 [ 0.5        -0.125      11.25       -4.8333335 ]
 [ 1.5        -0.125       9.75       -2.9251068 ]
 [ 1.25       -0.125      10.875      -9.25      ]]


In [140]:
cell = domain_tables.d_cells[p][i]
node_ghostnid = domain_tables.d_node_ghostid[p]
nodes = cell[0:cell[-1]]

print(node_ghostnid[nodes])

[[  2 441   0   0   0   0   2]
 [  5  65  73   0   0   0   3]
 [  2  15  19 441 444 456   6]
 [  0   0   0   0   0   0   0]]


In [137]:
print(domain_tables.d_cell_ghostnid[0][0:10])

[[  2 441   5  65  73  15  16  19 444 456   0   0   0   0   0   0   0  10]
 [  2   5  15  16  19 441 444 456  65  73  22  78  87   0   0   0   0  13]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2   5  15  16  19 441 444 456  22  73  78  87  31  32   0   0   0  14]
 [  2  15  19 441 444 456   5  16  22  73  78  87  31  32  35 459 470  17]
 [ 15  16  19  22  31  32  35 456 459 470   5  73  78  87  38  92 101  17]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]


In [118]:
print(domain_tables.d_cell_ghostnid[p].shape)

(1500, 25)


In [48]:
print(domain_tables.d_node_halonid[0])

[[  0   1   2 ...   0   0   9]
 [  1   3   4 ...   0   0   6]
 [  5   0   0 ...   0   0   1]
 ...
 [784 824 785 ...   0   0  10]
 [793 795 796 ...   0   0   9]
 [807 801 802 ...   0   0   9]]


In [68]:
def _is_in_array(array: 'int[:]', item: 'int') -> 'int':
  for i in range(array[-1]):
    if item == array[i]:
      return 1
  return 0

p = 0
cells = domain_tables.d_cells[p]
nb_cells = len(cells)
node_haloid = domain_tables.d_node_halonid[p]

cell_halonid = np.zeros(shape=(nb_cells, 20 + 1), dtype=np.int32)
for i in range(nb_cells):
  print(cells[i])
  for j in range(cells[i, -1]):
    n_halo = node_haloid[cells[i, j]]
    print("=>", n_halo)
    for k in range(n_halo[-1]):
      if _is_in_array(cell_halonid[i], n_halo[k]) == 0:
        size = cell_halonid[i, -1]
        cell_halonid[i, -1] += 1
        cell_halonid[i, size] = n_halo[k]
  print("Res: ", cell_halonid[i])
  if i == 2:
      break

# print(cell_halonid[0:10])

[0 1 2 3 4]
=> [ 0  1  2  3  4  5  8  9 10  0  0  0  0  0  0  0  0  0  0  0  0  0  9]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Res:  [ 0  1  2  3  4  5  8  9 10  6  7  0  0  0  0  0  0  0  0  0 11]
[2 3 1 4 4]
=> [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Res:  [5 1 3 4 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[0 1 3 5 4]
=> [ 0  1  2  3  4  5  8  9 10  0  0  0  0  0  0  0  0  0  0  0  0  0  9]
=> [1 3 4 5 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
=> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
=> [ 4  7 11 10 12 13 14 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8]
Res:  [ 0  1  2  3  4  5  8  9 10  6  7 11 12 13 14 15  0  0  0  0 16]


In [60]:
d_nodes = domain_tables.d_nodes[p]
d_cells = domain_tables.d_cells[p]
i = 400
c_cell_vertices = d_nodes[d_cells[i][0:d_cells[i][-1]]]
print(c_cell_vertices)

[[2.  1.  9. ]
 [3.  1.  9. ]
 [2.  1.5 7.5]
 [2.  1.5 9. ]]
